### Este es el ejemplo 4.3. Problema del Apostador (Gambler’s Problem) del libro de Sutton. 

Un apostador tiene la oportunidad de hacer apuestas a los resultados de una secuencia de tiros de una moneda. 
Si la moneda cae cara, gana tantos dólares como apostó en esa tirada.
Si la moneda cae ceca, pierde lo apostado. El juego termina cuando un apostador gana alcanzando su objetivo de $100, o pierde quedándose sin dinero.

En cada tirada el apostador debe decidir qué porción de su capital quiere apostar, una cantidad entera de dólares.
El problema puede ser formulado como un MDP finito sin descuento, episódico.

Los estados posibles del capital del apostador son: s ∈ {1, 2, . . . , 99}.

La acciones son apuestas,  a ∈ {0, 1, . . . , min(s, 100 − s)}. 

La recompensa es cero en todas las transiciones excepto en aquellas en que el apostador alcanza su objetivo, en que la recompensa es +1.

La función de estado-valor da la probabilidad de ganar desde cada estado. Una política es una función de niveles de capital a apuestas. La política óptima maximiza la probabilidad de obtener el objetivo. Llamemos p_h la probabilidad de que una moneda salga cara. Si p_h es conocida, entonces el problema se puede resolver, por ejemplo, con iteración de valor.

In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
if "../" not in sys.path:
  sys.path.append("../") 


### Exercise

Implementar iteración de valor para el problema del apostador y resolverlo para p_h = 0.25 y p_h = 0.55.


In [1]:
def value_iteration_for_gamblers(p_h, theta=0.0001, discount_factor=1.0):
    """
    Args:
        p_h: Probabilidad de que una moneda caiga cara
    """
 
    
    
    
    def one_step_lookahead(s, V, rewards):
        """
        Función auxiliar que calcula el valor de todas las acciones dado un estado.
        
        Args:
            s: El capital del apostador. Entero.
            V: El vector que contiene los valores en cada estado.
            rewards: El vector recompensa.
                        
        Returns:
            Un vector que contiene el valor esperado de cada acción.
            Su longitud es igual a la cantidad de acciones.
        """
        
        # Implementar!
        
        return A
    
    # Implementar!
    
    return policy, V

In [ ]:
policy, v = value_iteration_for_gamblers(0.25)

print("Política optimizada:")
print(policy)
print("")

print("Función de valor óptima:")
print(v)
print("")

In [ ]:
policy, v = value_iteration_for_gamblers(0.55)

print("Política optimizada:")
print(policy)
print("")

print("Función de valor óptima:")
print(v)
print("")

In [5]:
# Plotear la política final (apuesta) vs estado (capital)

# Implementar!

In [6]:
# Plotear capital vs política final

# Implementar!


# Scratchpad
En esta zona voy a ir armando los componentes necesarios para poder resolver el ejercicio, para luego pasarlo en limpio.

In [2]:
# constantes
n_S = 101
n_A = 51

### Politica

In [3]:
# empezamos con una random

def policy_random(n_S=101, n_A=51):
    policy = np.zeros([n_S, n_A])
    
    for s in range(51):
        for a in range(0, s+1):
            policy[s][a] = 1/(s+1)

    for s in range(51,101):
        n_a_posibles = min(s,100-s)+1
        for a in range(n_a_posibles):
            policy[s][a] = 1/(n_a_posibles)
    
    return policy

### Funcion de valor

In [4]:
V = np.zeros(n_S)

### env

In [5]:
# se respeta el formato utilizado en el ejercicio anterior, cada env[s][a] = lista de tuplas
# con formato (probabilidad, sig_estado, reward, done)

def reset_env(p=0.25):
    env = {}

    for s in range(0, 101):
        env[s] = {}

        for a in range(min(s,100-s)+1):        
            env[s][a] = []

            if (a==0):
                r = 1.0 if (s==100) else 0.0
                done = True if ((s==0) or (s==100)) else False
                env[s][a] = [(1.0, s, r, done)]
            else:
                sig_estado_si_gano = s+a
                sig_estado_si_pierdo = s-a
                done_si_gano = True if (sig_estado_si_gano==100) else False
                done_si_pierdo = True if (sig_estado_si_pierdo==0) else False
                r = 1.0 if (sig_estado_si_gano==100) else 0.0
                env[s][a] = [(p, sig_estado_si_gano, r, done_si_gano), ((1-p), (s-a), 0.0, done_si_pierdo)]
    return env

### Funciones

In [6]:
# policy_eval

def policy_eval(policy, env, n_S=101, discount_factor=1.0, theta=1e-05, n_max = 0):
    
    pasos = 0
    V = np.zeros(n_S)
    
    while True:
        
        delta = 0
        
        for s in range(1, n_S-1):
            v = 0
            for a in env[s]:
                for  prob, next_state, reward, done in env[s][a]:
                    v += policy[s][a] * prob * (reward + discount_factor * V[next_state])
            
            delta = max(delta, np.abs(v - V[s]))
            
            # for debug:
            #print(f"Paso {pasos+1} | Estado {s} | V_previo = {V[s]}, V_nuevo = {v}")
            
            
            V[s] = v
                
        pasos += 1
        
        if delta < theta:
            termino_correctamente = True
            break
            
        if (n_max):
            if (pasos == n_max):
                termino_correctamente = False
                break
    
    if (termino_correctamente):
        print(f"La eval. termino en {pasos} pasos.")
    else:
        print(f"Se alcanzaron los pasos maximos.")
   
    return np.array(V)

In [7]:
def policy_improvement(env, theta_opt=1e-08, theta_eval=1e-05, n_S=101, policy_eval_fn=policy_eval,
                       policy_seed_fn=policy_random, discount_factor=1.0, n_max=0, debug=False):
    
    pasos = 0
    
    # Comenzar con política aleatoria
    policy = policy_seed_fn()
    
    while True:
        
                
        V = policy_eval_fn(policy, env, theta= theta_eval, n_S=n_S, discount_factor=discount_factor)
                    
        politica_estable = True
        
        for s in range(1, n_S-1):
                        
            a_mas_probable = policy[s].tolist().index(max(policy[s]))
            # OJO, hay que pensar una manera de que cuando las p sean todas iguales, la cambie por 1.
            
            #print(f"state = {s}, a mas probable = {a_mas_probable}")
            
            acciones = []        
            for a in env[s]:
                retorno_a = 0
                for transicion in env[s][a]:
                    p_transicion = transicion[0]
                    proximo_estado = transicion[1]
                    reward = transicion[2]
                    retorno_a += p_transicion*(reward + discount_factor*V[proximo_estado])
                acciones.append((a, retorno_a))


            mejor_a = max(acciones, key=lambda x: x[1])[0]

            
            # si la acción de la política actual no coincide con la mejor calculada
            # actualizar la política
            # marcar que la política no fue estable en este paso
            diferencia = acciones[mejor_a][1] - acciones[a_mas_probable][1]
            #if ((diferencia > tetha) and (mejor_a != 0)):
                    
            if (diferencia > theta_opt):
                if (debug):
                    print(f"paso {pasos+1} | s = {s} | a_priori = {a_mas_probable} | dsp = {mejor_a}")
                    print(f"a_priori p = {acciones[a_mas_probable][1]} | dsp p = {acciones[mejor_a][1]}")
                politica_estable = False
                pol_s = np.zeros(51)
                pol_s[mejor_a] = 1
                policy[s] = pol_s
   
        # si la política es estable, devolver la política óptima y la función de valor de esa política
        if (politica_estable):
            print("Se encontro una politica optima.")
            return policy, V
        
        pasos += 1
        
        if (n_max):
            if (pasos == n_max):
                print("Se corta la optimizacion de la politica al llegar a los pasos maximos.")
                return policy, V

### Testeamos si funciona...

In [20]:
env = reset_env(p=0.25)
policy, v = policy_improvement(env, n_max=15, theta_eval=1e-3, theta_opt=1e-3)

La eval. termino en 22 pasos.
La eval. termino en 10 pasos.
La eval. termino en 10 pasos.
Se encontro una politica optima.


In [15]:
print("Política optimizada:")
print(policy)
print("")

print("Función de valor óptima:")
print(v)
print("")

Política optimizada:
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]

Función de valor óptima:
[0.00000000e+00 0.00000000e+00 1.83105469e-04 6.07490540e-04
 1.15966797e-03 1.60217285e-03 2.70247459e-03 3.90625000e-03
 4.63867188e-03 5.59997559e-03 7.01618195e-03 9.03320312e-03
 1.11241192e-02 1.56250000e-02 1.60806179e-02 1.68266296e-02
 1.85546875e-02 1.98249817e-02 2.23999023e-02 2.73437500e-02
 2.82449722e-02 3.04937363e-02 3.61328125e-02 3.84953022e-02
 4.44964767e-02 6.25000000e-02 6.26373291e-02 6.33697510e-02
 6.45268559e-02 6.59790039e-02 6.77621365e-02 7.08430894e-02
 7.45604634e-02 7.64160156e-02 7.92999268e-02 8.36837292e-02
 8.95996094e-02 9.58723575e-02 1.09477997e-01 1.10895142e-01
 1.13321602e-01 1.18420348e-01 1.21974945e-01 1.29699707e-01
 1.44608498e-01 1.47491202e-01 1.53981209e-01 1.70956373e-01
 1.77985907e-01 1.95989430e-01 2.50000000e-01 2.50137329e-01
 2.5

In [21]:
s = 0
for estado in policy:
    idx = estado.tolist().index(max(estado))
    if ((s != 0) and (s != 100)):
        print(f"Si tenes {s} dls, tenes que apostar {idx}. Tenes una prob de ganar del {round(100*v[s], 2)}%.")
        #print(f"Si tenes {s} dls, tenes que apostar {idx}. Tenes una prob de ganar del {100*v[s]}%.")
    s+= 1

Si tenes 1 dls, tenes que apostar 0. Tenes una prob de ganar del 0.0%.
Si tenes 2 dls, tenes que apostar 0. Tenes una prob de ganar del 0.02%.
Si tenes 3 dls, tenes que apostar 0. Tenes una prob de ganar del 0.04%.
Si tenes 4 dls, tenes que apostar 0. Tenes una prob de ganar del 0.09%.
Si tenes 5 dls, tenes que apostar 0. Tenes una prob de ganar del 0.14%.
Si tenes 6 dls, tenes que apostar 0. Tenes una prob de ganar del 0.21%.
Si tenes 7 dls, tenes que apostar 7. Tenes una prob de ganar del 0.4%.
Si tenes 8 dls, tenes que apostar 8. Tenes una prob de ganar del 0.47%.
Si tenes 9 dls, tenes que apostar 9. Tenes una prob de ganar del 0.56%.
Si tenes 10 dls, tenes que apostar 10. Tenes una prob de ganar del 0.71%.
Si tenes 11 dls, tenes que apostar 11. Tenes una prob de ganar del 0.9%.
Si tenes 12 dls, tenes que apostar 12. Tenes una prob de ganar del 1.11%.
Si tenes 13 dls, tenes que apostar 13. Tenes una prob de ganar del 1.57%.
Si tenes 14 dls, tenes que apostar 14. Tenes una prob de ga